### 待改进部分
 1 过拟合 调参  
 2 数据不均衡：（1）使用下采样（2）使用auc分数

In [1]:
import numpy as np
import pandas as pd
import pymongo
import tensorflow as tf
import os
import time
day=time.strftime("%Y-%m-%d", time.localtime())

In [2]:
#从数据库中读取数据
client=pymongo.MongoClient('localhost',27017)#连接数据库
db1=client['douban']#创建新数据库
pred_comments=db1['pred_comments']
data = pd.DataFrame(list(pred_comments.find()))
data.head(2)

,_id,content,processed_comment,star
0,5cdbd2cda1cf18259c85b40e,当然算不上杰作，但放豆瓣语境下，是部时至今日终于拍出来的国产“高分韩国电影”。拿现实题材拍商...,"[算不上, 杰作, 但放, 豆瓣, 语境, 时至今日, 终于, 国产, 高分, 韩国, 电影...",4
1,5cdbd2cda1cf18259c85b40f,最大的病，其实是穷病。真的被感动了，整体都很成熟，也有些许韩片的影子。几个演员表演都非常出色...,"[最大, 其实, 穷病, 真的, 感动, 整体, 成熟, 些许, 韩片, 影子, 几个, 演...",4


In [3]:
word_vectors=np.load('sample_Tencent_AILab_ChineseEmbedding.npy',allow_pickle=True).item()
#转换词向量
def convert_to_vec(words_list):
    words_vec=[]
    for i in words_list:
        if i in word_vectors:
            words_vec.append(np.array(word_vectors[i]))
    return np.array(words_vec)
#获取每个评论词向量的数量
def get_seq_length(data):
    return len(data)
# 转换评星
def convert_stars(star):
    if int(star)>3:
        return 1
    else:
        return 0

data['word_vec']=data.processed_comment.apply(convert_to_vec)
data['pos1neg0']=data.star.apply(convert_stars)
data['seq_length']=data.word_vec.apply(get_seq_length)
data=data[[list(i)!=[] for i in data['word_vec']]]#删除词向量为[]的文本
data.head(2)

,_id,content,processed_comment,star,word_vec,pos1neg0,seq_length
0,5cdbd2cda1cf18259c85b40e,当然算不上杰作，但放豆瓣语境下，是部时至今日终于拍出来的国产“高分韩国电影”。拿现实题材拍商...,"[算不上, 杰作, 但放, 豆瓣, 语境, 时至今日, 终于, 国产, 高分, 韩国, 电影...",4,"[[0.192524, -0.185849, 0.275798, 0.053039, 0.5...",1,27
1,5cdbd2cda1cf18259c85b40f,最大的病，其实是穷病。真的被感动了，整体都很成熟，也有些许韩片的影子。几个演员表演都非常出色...,"[最大, 其实, 穷病, 真的, 感动, 整体, 成熟, 些许, 韩片, 影子, 几个, 演...",4,"[[-0.026906, 0.003273, 0.084483, 0.253033, 0.0...",1,30


In [4]:
data.describe()

,star,pos1neg0,seq_length
count,83247.000000,83247.000000,83247.000000
mean,4.028289,0.845075,16.523875
std,1.126239,0.361835,14.875728
min,1.000000,0.000000,1.000000
25%,4.000000,1.000000,5.000000
50%,4.000000,1.000000,12.000000
75%,5.000000,1.000000,25.000000
max,5.000000,1.000000,930.000000


In [5]:
num2=np.mean(data[:].processed_comment.apply(lambda x:len(x)))
num1=np.mean(data[:].word_vec.apply(lambda x:len(x)))
print('平均每个评论有',num2,'个词')
print('平均每个评论有',num1,'个 wordvec')

平均每个评论有 17.11447859982942 个词
平均每个评论有 16.523874734224655 个 wordvec


In [6]:
#下采样
negdata=data[~data['pos1neg0'].isin([1])]
posdata=data[~data['pos1neg0'].isin([0])]
xiacaiyangpos=posdata.sample(frac=0.25,replace=False)
newdata=pd.concat( [negdata,xiacaiyangpos], axis=0 ) 
data=newdata

In [8]:
X=np.array(data['word_vec'])
num=30#要保留的维度，不足补0
X_new=np.zeros((len(X),num,200))
for i,j in enumerate(X):
    if j.shape[0]>=num:
        X_new[i]=X[i][:num]
    else:
        X_new[i]=np.concatenate([X[i],np.zeros(((num-j.shape[0]),200))])       
X=X_new
y=np.array(data['pos1neg0'])
seq=np.array(data['seq_length'])
# for i,j in enumerate(seq):
#     if j>num:
#         seq[i]=num

print('数据总数量：',y.shape[0])
print('负类所占比例：',list(y).count(0)/(list(y).count(1)+list(y).count(0)))
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test,seq_train,seq_test=train_test_split(X,np.array(y),seq,test_size=0.2)
print('训练集数量：',y_train.shape[0])
print('测试集数量：',y_test.shape[0])
print('训练集负类所占比例：',list(y_train).count(0)/(list(y_train).count(1)+list(y_train).count(0)))
print('测试集负类所占比例：',list(y_test).count(0)/(list(y_test).count(1)+list(y_test).count(0)))

数据总数量： 30485
负类所占比例： 0.42306052156798424
训练集数量： 24388
测试集数量： 6097
训练集负类所占比例： 0.4230769230769231
测试集负类所占比例： 0.42299491553222895


In [9]:
def shuffle_batch(X, y, seq, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch,seq_batch = X[batch_idx], y[batch_idx],seq[batch_idx]
        yield X_batch, y_batch,seq_batch

In [10]:
tf.reset_default_graph()

n_steps = num
n_inputs=200
n_neurons = 64
n_outputs = 2

X = tf.placeholder(tf.float32, [None, None,n_inputs],name='X')
y = tf.placeholder(tf.int32, [None],name='y')
# seq_length = tf.placeholder(tf.int32, [None], name="seq_length")

with tf.name_scope('RNN'):
    lstmCell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
    # 配置dropout参数，以此避免过拟合
    droplstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
    outputs, states = tf.nn.dynamic_rnn(droplstmCell, X, dtype=tf.float32)#,sequence_length=seq_length)
    
with tf.name_scope('Loss'):
    logits = tf.layers.dense(outputs[:,-1,:],n_outputs)
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(xentropy)
    loss_summary = tf.summary.scalar('log_loss', loss)#使用tensorboard

learning_rate = 0.001
with tf.name_scope('Train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
with tf.name_scope("Eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer = tf.summary.FileWriter('douban_log/'+day, tf.get_default_graph())


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


In [ ]:
#定义mini-batch大小与循环轮次
n_epochs = 200
batch_size = 100
n_batches = int(np.ceil(X_train.shape[0] / batch_size))

#保存检查点
checkpoint_path = "tmp_model/"+day+"/my_douban_rnn_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "model/"+day+"/my_douban_rnn_model"

#如果出现50次loss大于之前最好的损失，提前停止
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 20

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
        
    for epoch in range(start_epoch, n_epochs):
        for X_batch, y_batch,seq_batch in shuffle_batch(X_train, y_train, seq, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})#, seq_length:seq_batch})
        
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run(
            [accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: X_batch, y: y_batch}#, seq_length:seq_batch}
        )
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})#,seq_length:seq_test})
        print("Epoch:", epoch,"\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),"\tLoss: {:.5f}".format(loss_val),
             "\tTest accuracy: {:.3f}%".format(acc_test * 100))
        saver.save(sess, checkpoint_path)
        with open(checkpoint_epoch_path, "wb") as f:
            f.write(b"%d" % (epoch + 1))
        if loss_val < best_loss:
            saver.save(sess, final_model_path)
            best_loss = loss_val
        else:
            epochs_without_progress += 1
            if epochs_without_progress > max_epochs_without_progress:
                print("Early stopping")
                break

Epoch: 0 	Validation accuracy: 73.000% 	Loss: 0.56698 	Test accuracy: 77.612%
Epoch: 1 	Validation accuracy: 76.000% 	Loss: 0.46372 	Test accuracy: 75.513%
Epoch: 2 	Validation accuracy: 79.000% 	Loss: 0.44231 	Test accuracy: 79.482%
Epoch: 3 	Validation accuracy: 81.000% 	Loss: 0.43146 	Test accuracy: 78.760%
Epoch: 4 	Validation accuracy: 76.000% 	Loss: 0.46080 	Test accuracy: 79.449%
Epoch: 5 	Validation accuracy: 82.000% 	Loss: 0.36151 	Test accuracy: 80.384%
Epoch: 6 	Validation accuracy: 80.000% 	Loss: 0.53343 	Test accuracy: 81.450%
Epoch: 7 	Validation accuracy: 83.000% 	Loss: 0.44729 	Test accuracy: 80.417%
Epoch: 8 	Validation accuracy: 88.000% 	Loss: 0.25400 	Test accuracy: 80.761%
Epoch: 9 	Validation accuracy: 89.000% 	Loss: 0.32133 	Test accuracy: 80.433%
Epoch: 10 	Validation accuracy: 92.000% 	Loss: 0.23793 	Test accuracy: 80.892%
Epoch: 11 	Validation accuracy: 89.000% 	Loss: 0.22683 	Test accuracy: 80.663%
Epoch: 12 	Validation accuracy: 84.000% 	Loss: 0.33554 	Test a

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
accuracy_val

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    logits1 = logits.eval(feed_dict={X: X_train, y: y_train})
    logits2 = logits.eval(feed_dict={X: X_test, y: y_test})
from sklearn.metrics import confusion_matrix
y_train_pre=[0 if i[0]>i[1] else 1 for i in logits1]
y_test_pre=[0 if i[0]>i[1] else 1 for i in logits2]
print('训练集: \n',confusion_matrix(y_train,y_train_pre))
print('测试集: \n',confusion_matrix(y_test,y_test_pre))

In [ ]:
final_model_path = "model/my_douban_rnn_model"
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    output = outputs.eval(feed_dict={X: X_train, y: y_train})
#     states = states.eval(feed_dict={X: X_train, y: y_train})

In [ ]:
states